# Extended Subgraph Partitioning (orb-models)

In [106]:
import numpy as np
import torch

from tqdm import tqdm

### Extended Partitioning

In [107]:
import networkx as nx
import metis

from collections import deque

def part_graph_extended(G, desired_partitions, distance=None):
    def descendants_at_distance_multisource(G, sources, distance=None):
        if sources in G:
            sources = [sources]

        queue = deque(sources)
        depths = deque([0 for _ in queue])
        visited = set(sources)

        for source in queue:
            if source not in G:
                raise nx.NetworkXError(f"The node {source} is not in the graph.")

        while queue:
            node = queue[0]
            depth = depths[0]

            if distance is not None and depth > distance: return

            yield queue[0]

            queue.popleft()
            depths.popleft()

            for child in G[node]:
                if child not in visited:
                    visited.add(child)
                    queue.append(child)
                    depths.append(depth + 1)

    _, parts = metis.part_graph(G, desired_partitions, objtype="cut")
    partition_map = {node: parts[i] for i, node in enumerate(G.nodes())}
    num_partitions = desired_partitions

    # Find indices of nodes in each partition
    partitions = [set() for _ in range(desired_partitions)]

    for i, node in enumerate(G.nodes()):
        partitions[partition_map[i]].add(node)

    # Find boundary nodes (vertices adjacent to vertex not in partition)
    boundary_nodes = [set(map(lambda uv: uv[0], nx.edge_boundary(G, partitions[i]))) for i in range(num_partitions)]

    # Perform BFS on boundary_nodes to find extended neighbors up to a certain distance
    extended_neighbors = [set(descendants_at_distance_multisource(G, boundary_nodes[i], distance=distance)) for i in range(num_partitions)]

    extended_partitions = [p.union(a) for p, a in zip(partitions, extended_neighbors)]

    return partitions, extended_partitions

## Partitioning Atoms

Loading a sample atomic dataset and converting it into a graph

In [108]:
from ase.io import read
from orb_models.forcefield.atomic_system import ase_atoms_to_atom_graphs
from ase.build import make_supercell
 
atoms = read("datasets/test.xyz")
atoms = make_supercell(atoms, [[2, 0, 0], [0, 2, 0], [0, 0, 2]])

# Instead of using neighborlist, I use the ase_atoms_to_atom_graphs provided by orb. Hopefully this will provide better results
atom_graph = ase_atoms_to_atom_graphs(atoms) # Keep this to compare results later

senders = atom_graph.senders
receivers = atom_graph.receivers
edge_feats = atom_graph.edge_features

G = nx.Graph()
G.add_nodes_from(range(len(atoms)))

for i, u in enumerate(senders):
    G.add_edge(u.item(), receivers[i].item(), weight=edge_feats['r'])

print("Number of atoms", len(atoms))

Number of atoms 3408


Partition the computational graph into the number of desired partitions with the specified neighborhood distance

In [109]:
desired_partitions = 20
neighborhood_distance = 5
partitions, extended_partitions = part_graph_extended(G, desired_partitions, neighborhood_distance)

num_partitions = len(partitions)

Create the ASE atoms object for each partition

In [110]:
from ase import Atoms

partitioned_atoms = []
indices_map = [] # Table mapping each atom in each partition back to its index in the original atoms object

for part in extended_partitions:
    current_partition = []
    current_indices_map = []
    for atom_index in part:
        current_partition.append(atoms[atom_index])
        current_indices_map.append(atoms[atom_index].index)

    partitioned_atoms.append(Atoms(current_partition, cell=atoms.cell, pbc=atoms.pbc)) # It's important to pass atoms.cell and atoms.pbc here
    indices_map.append(current_indices_map)

In [111]:
reconstructed_atoms = []
for atom_index in range(len(atoms)):
    reconstructed_atoms.append(atoms[atom_index])
reconstructed_atoms = Atoms(reconstructed_atoms, cell=atoms.cell, pbc=atoms.pbc)

reconstructed_atoms

Atoms(symbols='C880H2208Ga64S64Si192', pbc=True, cell=[[23.096664428710938, 13.334883689880371, 23.9624080657959], [-23.09648895263672, 13.334826469421387, 23.962270736694336], [3.831999856629409e-05, -26.669597625732422, 23.962278366088867]])

## Inference

In [112]:
from orb_models.forcefield import atomic_system, pretrained
from orb_models.forcefield import segment_ops

Load model

In [113]:
device = "cpu"  # or device="cuda"

orbff = pretrained.orb_v2(device=device)

/opt/anaconda3/envs/orb-partitioning/lib/python3.10/site-packages/orb_models/forcefield/pretrained.py:71: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.lo

Run inference on each partition

In [114]:
aggregated_features = torch.zeros((len(atoms), 256), dtype=torch.float32, device=device)

for i, part in tqdm(enumerate(partitioned_atoms), total=num_partitions):
    input_graph = atomic_system.ase_atoms_to_atom_graphs(part)

    batch = orbff.model(input_graph)

    feat = batch.node_features["feat"]

    for j, node in enumerate(part):
        original_index = indices_map[i][j]
        if original_index in partitions[i]: # If the node is a root node of the partition
            aggregated_features[original_index] = feat[j]

aggregated_features

100%|██████████| 20/20 [03:27<00:00, 10.37s/it]


tensor([[-0.3629,  0.6121, -0.2159,  ..., -0.1818,  0.5214,  0.2927],
        [-0.1475,  0.8231,  0.1479,  ..., -0.0515,  0.2717, -0.3499],
        [-0.5161,  0.6599, -0.2023,  ...,  0.0230,  0.3217, -0.8838],
        ...,
        [ 0.4505, -0.0583,  0.0022,  ...,  0.0957, -0.1776, -0.0512],
        [ 0.7821,  0.0331,  0.2452,  ..., -0.0250, -0.0294, -0.4117],
        [ 0.8261, -0.0955,  0.0826,  ...,  0.1450, -0.5848, -0.3101]])

## Prediction

Run the prediction on the original, unpartitioned graph to obtain a benchmark for our results

In [115]:
result = orbff.predict(atom_graph)
benchmark_energy = result["graph_pred"]
benchmark_forces = result["node_pred"]
benchmark_stress = result["stress_pred"]

In [116]:
from orb_models.forcefield.graph_regressor import ScalarNormalizer, LinearReferenceEnergy
from orb_models.forcefield.reference_energies import REFERENCE_ENERGIES

ref = REFERENCE_ENERGIES["vasp-shifted"]
reference = LinearReferenceEnergy(
    weight_init=ref.coefficients, trainable=True
)

n_node = torch.tensor([aggregated_features.shape[0]])

### Energy

In [117]:
input = segment_ops.aggregate_nodes(
    aggregated_features,
    n_node,
    reduction="mean"
)

energy = orbff.graph_head.mlp(input)
energy = orbff.graph_head.normalizer.inverse(energy).squeeze(-1)
energy = energy * n_node
energy = energy + reference(atom_graph.atomic_numbers, atom_graph.n_node)
energy

tensor([-17688.2305], grad_fn=<AddBackward0>)

In [123]:
print(f"Absolute error: {torch.abs(benchmark_energy - energy).item()}")
print(f"Percent error: {torch.abs((benchmark_energy - energy) / benchmark_energy).item() * 100}%")
print(f"Maximum error: {torch.max(benchmark_energy - energy).item()}")

Absolute error: 0.0
Percent error: 0.0%
Maximum error: 0.0


### Forces

In [119]:
forces = orbff.node_head.mlp(aggregated_features)
system_means = segment_ops.aggregate_nodes(
    forces, n_node, reduction="mean"
)
node_broadcasted_means = torch.repeat_interleave(
    system_means, n_node, dim=0
)
forces = forces - node_broadcasted_means
forces = orbff.node_head.normalizer.inverse(forces)
forces

tensor([[ 0.0349, -0.0312,  0.0005],
        [-0.0958, -0.0764, -0.0108],
        [-0.0571,  0.0508,  0.0085],
        ...,
        [ 0.0123, -0.1079,  0.0407],
        [-0.1089,  0.0325,  0.0415],
        [ 0.0791,  0.0703,  0.0156]])

In [127]:
mae = torch.mean(torch.abs(benchmark_forces - forces))
mape = 100 * torch.mean(torch.abs((benchmark_forces - forces) / benchmark_forces))
max = torch.max(torch.abs(benchmark_forces - forces))

print(f"Mean absolute error: {mae.item()}")
print(f"Mean absolute percent error: {mape}%")
print(f"Maximum error: {max}")

Mean absolute error: 6.320112220237206e-07
Mean absolute percent error: 0.0020459508523344994%
Maximum error: 0.0004204772412776947


### Stress

In [121]:
input = segment_ops.aggregate_nodes(
    aggregated_features,
    n_node,
    reduction="mean",
)
stress = orbff.stress_head.mlp(input)
stress = stress.squeeze(-1)
stress = orbff.stress_head.output_activation(stress)
stress = orbff.stress_head.normalizer.inverse(stress)
stress

tensor([[ 1.1085e-02,  1.1161e-02,  1.2405e-02,  8.0176e-06, -1.8518e-06,
         -2.5269e-06]])

In [125]:
mae = torch.mean(torch.abs(benchmark_stress - stress))
mape = 100 * torch.mean(torch.abs((benchmark_stress - stress) / benchmark_stress))
max = torch.max(torch.abs(benchmark_stress - stress))
print(f"Mean absolute error: {mae.item()}")
print(f"Mean absolute percent error: {mape}%")
print(f"Max error: {max}")

Mean absolute error: 1.1397060006856918e-07
Mean absolute percent error: 0.002338955644518137%
Max error: 2.3655593395233154e-07
